In [31]:
from bisect import bisect

from countryinfo import CountryInfo
import numpy as np
import pandas as pd

In [86]:
def death_stat_raw():
    url = 'https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_deaths_global.csv&filename=time_series_covid19_deaths_global.csv'
    return pd.read_csv(url)

def raw_data():
    stats = death_stat_raw()
    stats.head()

    china = stats.loc[(stats['Country/Region'] == 'China')].sum()
    china['Country/Region'] = 'China'
    china['Province/State'] = np.nan
    stats = stats.append(china, ignore_index=True)
    country = stats[stats['Province/State'].isnull()]

    del country['Province/State']
    del country['Lat']
    del country['Long']
    
    return country

def death_per_country():
    data = raw_data()
    countries = {}
    for idx, row in data.iterrows():
        for n, (k, v) in enumerate(row.items()):
            if n == 0:
                if v == 'US':
                    v = 'United States'
                elif v == 'Korea, South':
                    v = 'South Korea'
                data = countries[v] = []
            else:
                data.append(v)

    # remove zeros and calculate deaths per day
    # starting from the first day with 5 deaths
    no_data = []
    deaths = {}
    for country, v in countries.items():
        v = v[max(bisect(v, 5)-1, 0):]

        if len(v) < 10 or v[-1] < 100:
            continue

        res = []
        for a, b in zip(v, v[1:]):
            res.append(b-a)

        cinfo = CountryInfo(country)
        population = cinfo.population() / 1_000_000

        dayly_deaths = np.array(res) / population
        dayly_deaths[dayly_deaths==0] = np.nan
        dayly_deaths = moving_average(dayly_deaths)
        all_deaths = moving_average(np.array(v[1:]) / population)

        deaths[country] = {'per_day': dayly_deaths, 'cumulatif': all_deaths}
    return deaths

def moving_average(a, n=5) :
    ret = np.nancumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
# %matplotlib notebook

deaths = death_per_country()

d = {
    'France': deaths['France'],
    'Italy': deaths['Italy'],
#     'United States': deaths['United States'],
    'Iran': deaths['Iran'],
    'United Kingdom': deaths['United Kingdom'],
    'Belgium': deaths['Belgium'],
    'China': deaths['China'],
    'South Korea': deaths['South Korea']
}

legend = []
for c, data in d.items():
    legend.append(c)
    cinfo = CountryInfo(c)
    population = cinfo.population() / 1_000_000

    y = data['per_day'] / population
    y[y==0] = np.nan
    y = moving_average(y)
    x = moving_average(data['cumulatif'] / population)
    plt.plot(x, y)

plt.grid(alpha=0.4)
plt.legend(legend, title='Country')
plt.xscale('log')
plt.xlabel('Deaths per million people')
plt.yscale('log')
plt.ylabel('Deaths per day per million people')

ax = plt.gca()
ax.xaxis.set_major_formatter(ScalarFormatter())
ax.yaxis.set_major_formatter(ScalarFormatter())
plt.show()

In [171]:
%config InlineBackend.figure_formats = ['svg']
# %matplotlib notebook
%matplotlib widget

import ipywidgets as widgets
from IPython.display import HTML
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
plt.style.use('ggplot')


HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")


debugoutput = widgets.Output()


class App:
    DEFAULT_COUNTRY = 'Germany'

    def __init__(self):
        self.deaths = death_per_country()
        countries = sorted(self.deaths)
        self.shown = {}
        
        self._country_dropdown = self.create_dropdown(countries, countries.index(self.DEFAULT_COUNTRY))
        self._plot_container = widgets.Output()
        global debugoutput
        self._debug = debugoutput

        _app_container = widgets.VBox(
            [
                widgets.HBox(
                    [widgets.HTML(('<b1>Select country to plot:</b1>')), self._country_dropdown]),
                self._plot_container,
                self._debug
            ],
            layout=widgets.Layout(align_items='center', flex='2 0 auto')
        )
        self.container = widgets.VBox(
            [
                widgets.HTML(('<h1>Coronavirus Country Status</h1>'), 
                             layout=widgets.Layout(margin='0 0 5em 0')),
                widgets.HBox([
                    _app_container,
                ])
            ],
            layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px')
        )
        self.update(self.DEFAULT_COUNTRY)

    def create_dropdown(self, indicators, initial_index=0):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown

    @debugoutput.capture(clear_output=True)
    def plot(self, country):
        ax = plt.gca()

        if country in self.shown:
            # remove line for country
            country_line = self.shown.pop(country)
            ax.lines.remove(country_line)
            return

        legend = list(self.shown) + [country]
        deaths = self.deaths[country]
        ax.plot(deaths['cumulatif'], deaths['per_day'])

        ax.grid(alpha=0.4)
        ax.legend(legend, title='Country')
        plt.xscale('log')
        plt.xlabel('Deaths per million people')
        plt.yscale('log')
        plt.ylabel('Deaths per day per million people')

        ax.xaxis.set_major_formatter(ScalarFormatter())
        ax.yaxis.set_major_formatter(ScalarFormatter())
        
        self.shown[country] = ax.lines[-1]

    def _on_change(self, dropbox):
        self.update(dropbox['new'])

    @debugoutput.capture(clear_output=True)
    def update(self, country):
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self.plot(country)
            plt.show()

In [172]:
app = App()

app.container